In [13]:
import requests
import os
from bs4 import BeautifulSoup
from PyPDF2 import PdfMerger
import time
import logging
import urllib.parse
import subprocess

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# URL de la página principal
main_url = "https://madi.nekomath.com/P5/ReduccionesPvsNP.html"

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        logger.info(f"Contenido de la página: {response.text[:500]}...")  # Mostrar los primeros 500 caracteres
        soup = BeautifulSoup(response.text, 'html.parser')
        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            logger.info(f"Enlace encontrado: {href}")
            if href.startswith(('Satisfacibilidad', 'CliqueCoverVertex', 'HamiltonTSP', '3COLOR')) or href == "ReduccionesPvsNP.html":
                full_url = urllib.parse.urljoin(url, href)
                if full_url not in links:
                    links.append(full_url)
        logger.info(f"Total de enlaces relevantes encontrados: {len(links)}")
        return links
    except requests.exceptions.RequestException as e:
        logger.error(f"Error al obtener enlaces de la página principal: {e}")
        return []

def html_to_pdf(url, output_filename):
    try:
        command = ['wkhtmltopdf', url, output_filename]
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        logger.info(f"Salida de wkhtmltopdf: {result.stdout}")
        logger.info(f"PDF creado con éxito: {output_filename}")
        return True
    except subprocess.CalledProcessError as e:
        logger.error(f"Error al crear PDF {output_filename}: {e}")
        logger.error(f"Salida de error: {e.stderr}")
        return False

# Crear directorio para guardar los PDFs
if not os.path.exists("pdfs"):
    os.makedirs("pdfs")

# Obtener enlaces de la página principal
page_links = get_page_links(main_url)
logger.info(f"Enlaces obtenidos: {page_links}")

# Procesar cada página
successful_pdfs = []
for page_url in page_links:
    logger.info(f"Procesando página: {page_url}")
    
    pdf_filename = f"pdfs/{os.path.basename(page_url).replace('.html', '.pdf')}"
    if html_to_pdf(page_url, pdf_filename):
        successful_pdfs.append(pdf_filename)
    
    time.sleep(1)  # Esperar 1 segundo entre solicitudes

# Combinar PDFs
if successful_pdfs:
    merger = PdfMerger()
    for pdf in successful_pdfs:
        try:
            merger.append(pdf)
            logger.info(f"PDF agregado con éxito: {pdf}")
        except Exception as e:
            logger.error(f"Error al procesar {pdf}: {e}")

    try:
        merger.write("madi_nekomath_combined.pdf")
        merger.close()
        logger.info("PDFs combinados con éxito en 'madi_nekomath_combined.pdf'")
    except Exception as e:
        logger.error(f"Error al escribir el PDF combinado: {e}")
else:
    logger.warning("No se crearon PDFs con éxito. No se puede combinar.")

logger.info("Proceso completado. Por favor, revisa los logs para más detalles.")

INFO:__main__:Contenido de la página: 

<!DOCTYPE html>


<html lang="es" >

  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="generator" content="Docutils 0.18.1: http://docutils.sourceforge.net/" />

    <title>Clases de complejidad y P vs. NP &#8212; MatemÃ¡ticas Discretas para Ciencia de Datos</title>
  
  
  
  <script data-cfasync="false">
    document.documentElement.dataset.mode = localStorage.getItem("mode") || "";
    document.documentElement.datas...
INFO:__main__:Enlace encontrado: #main-content
INFO:__main__:Enlace encontrado: ../intro.html
INFO:__main__:Enlace encontrado: ../intro.html
INFO:__main__:Enlace encontrado: ../P1/IntroCombinatoria.html
INFO:__main__:Enlace encontrado: ../P1/BuscarPatron.html
INFO:__main__:Enlace encontrado: ../P1/Casillas.html
INFO:__main__:Enlace encontrado: ../P1/DobleConteo.html
INFO:__main__:Enlace encontrado: ../P1/Induccion.html
INFO:__main__:Enlace encontrado: ..

In [14]:
import requests
import os
from bs4 import BeautifulSoup
from PyPDF2 import PdfMerger
import time
import logging
import urllib.parse
import subprocess

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# URL de la página principal
main_url = "https://madi.nekomath.com/P5/ReduccionesPvsNP.html"

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        logger.info(f"Contenido de la página: {response.text[:500]}...")
        soup = BeautifulSoup(response.text, 'html.parser')
        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            logger.info(f"Enlace encontrado: {href}")
            if href.endswith('.html') and not href.startswith(('#', 'http', 'https')):
                full_url = urllib.parse.urljoin(url, href)
                if full_url not in links:
                    links.append(full_url)
        logger.info(f"Total de enlaces relevantes encontrados: {len(links)}")
        return links
    except requests.exceptions.RequestException as e:
        logger.error(f"Error al obtener enlaces de la página principal: {e}")
        return []

def html_to_pdf(url, output_filename):
    try:
        # Paso 1: Descargar el HTML
        response = requests.get(url)
        response.raise_for_status()
        html_content = response.text

        # Paso 2: Convertir HTML a LaTeX usando Pandoc
        latex_filename = output_filename.replace('.pdf', '.tex')
        pandoc_html_to_latex = ['pandoc', '-f', 'html', '-t', 'latex', '--mathjax', '-o', latex_filename]
        result = subprocess.run(pandoc_html_to_latex, input=html_content, text=True, capture_output=True, check=True)
        logger.info(f"LaTeX creado con éxito: {latex_filename}")

        # Paso 3: Convertir LaTeX a PDF
        pdflatex_command = ['pdflatex', '-interaction=nonstopmode', latex_filename]
        result = subprocess.run(pdflatex_command, check=True, capture_output=True, text=True)
        logger.info(f"PDF creado con éxito: {output_filename}")

        # Limpiar archivos temporales
        for ext in ['.aux', '.log', '.out']:
            temp_file = output_filename.replace('.pdf', ext)
            if os.path.exists(temp_file):
                os.remove(temp_file)

        return True
    except (subprocess.CalledProcessError, requests.exceptions.RequestException) as e:
        logger.error(f"Error al crear PDF {output_filename}: {e}")
        return False

# Crear directorio para guardar los PDFs
if not os.path.exists("pdfs"):
    os.makedirs("pdfs")

# Obtener enlaces de la página principal
page_links = get_page_links(main_url)
logger.info(f"Enlaces obtenidos: {page_links}")

# Procesar cada página
successful_pdfs = []
for page_url in page_links:
    logger.info(f"Procesando página: {page_url}")
    
    pdf_filename = f"pdfs/{os.path.basename(page_url).replace('.html', '.pdf')}"
    if html_to_pdf(page_url, pdf_filename):
        successful_pdfs.append(pdf_filename)
    
    time.sleep(1)  # Esperar 1 segundo entre solicitudes

# Combinar PDFs
if successful_pdfs:
    merger = PdfMerger()
    for pdf in successful_pdfs:
        try:
            merger.append(pdf)
            logger.info(f"PDF agregado con éxito: {pdf}")
        except Exception as e:
            logger.error(f"Error al procesar {pdf}: {e}")

    try:
        merger.write("madi_nekomath_combined.pdf")
        merger.close()
        logger.info("PDFs combinados con éxito en 'madi_nekomath_combined.pdf'")
    except Exception as e:
        logger.error(f"Error al escribir el PDF combinado: {e}")
else:
    logger.warning("No se crearon PDFs con éxito. No se puede combinar.")

logger.info("Proceso completado. Por favor, revisa los logs para más detalles.")

INFO:__main__:Contenido de la página: 

<!DOCTYPE html>


<html lang="es" >

  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="generator" content="Docutils 0.18.1: http://docutils.sourceforge.net/" />

    <title>Clases de complejidad y P vs. NP &#8212; MatemÃ¡ticas Discretas para Ciencia de Datos</title>
  
  
  
  <script data-cfasync="false">
    document.documentElement.dataset.mode = localStorage.getItem("mode") || "";
    document.documentElement.datas...
INFO:__main__:Enlace encontrado: #main-content
INFO:__main__:Enlace encontrado: ../intro.html
INFO:__main__:Enlace encontrado: ../intro.html
INFO:__main__:Enlace encontrado: ../P1/IntroCombinatoria.html
INFO:__main__:Enlace encontrado: ../P1/BuscarPatron.html
INFO:__main__:Enlace encontrado: ../P1/Casillas.html
INFO:__main__:Enlace encontrado: ../P1/DobleConteo.html
INFO:__main__:Enlace encontrado: ../P1/Induccion.html
INFO:__main__:Enlace encontrado: ..

In [20]:
import requests
import os
from bs4 import BeautifulSoup
from PyPDF2 import PdfMerger
import time
import logging
import urllib.parse
import subprocess

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# URL de la página principal
main_url = "https://madi.nekomath.com/P5/ReduccionesPvsNP.html"

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        logger.info(f"Contenido de la página: {response.text[:500]}...")
        soup = BeautifulSoup(response.text, 'html.parser')
        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            logger.info(f"Enlace encontrado: {href}")
            if href.endswith('.html') and not href.startswith(('#', 'http', 'https')):
                full_url = urllib.parse.urljoin(url, href)
                if full_url not in links:
                    links.append(full_url)
        logger.info(f"Total de enlaces relevantes encontrados: {len(links)}")
        return links
    except requests.exceptions.RequestException as e:
        logger.error(f"Error al obtener enlaces de la página principal: {e}")
        return []

def html_to_pdf(url, output_filename):
    try:
        # Paso 1: Descargar el HTML
        response = requests.get(url)
        response.raise_for_status()
        html_content = response.text

        # Paso 2: Convertir HTML a LaTeX usando Pandoc
        latex_filename = output_filename.replace('.pdf', '.tex')
        pandoc_html_to_latex = ['pandoc', '-f', 'html', '-t', 'latex', '--mathjax', '-o', latex_filename]
        result = subprocess.run(pandoc_html_to_latex, input=html_content, text=True, capture_output=True, check=True)
        logger.info(f"LaTeX creado con éxito: {latex_filename}")

        # Paso 3: Convertir LaTeX a PDF
        pdflatex_command = ['pdflatex', '-interaction=nonstopmode', latex_filename]
        result = subprocess.run(pdflatex_command, check=True, capture_output=True, text=True)
        logger.info(f"PDF creado con éxito: {output_filename}")

        # Limpiar archivos temporales
        for ext in ['.aux', '.log', '.out']:
            temp_file = output_filename.replace('.pdf', ext)
            if os.path.exists(temp_file):
                os.remove(temp_file)

        return True
    except (subprocess.CalledProcessError, requests.exceptions.RequestException) as e:
        logger.error(f"Error al crear PDF {output_filename}: {e}")
        return False

# Crear directorio para guardar los PDFs
if not os.path.exists("pdfs"):
    os.makedirs("pdfs")

# Obtener enlaces de la página principal
page_links = get_page_links(main_url)
logger.info(f"Enlaces obtenidos: {page_links}")

# Procesar cada página
successful_pdfs = []
for page_url in page_links:
    logger.info(f"Procesando página: {page_url}")
    
    pdf_filename = f"pdfs/{os.path.basename(page_url).replace('.html', '.pdf')}"
    if html_to_pdf(page_url, pdf_filename):
        successful_pdfs.append(pdf_filename)
    
    time.sleep(1)  # Esperar 1 segundo entre solicitudes

# Combinar PDFs
if successful_pdfs:
    merger = PdfMerger()
    for pdf in successful_pdfs:
        try:
            merger.append(pdf)
            logger.info(f"PDF agregado con éxito: {pdf}")
        except Exception as e:
            logger.error(f"Error al procesar {pdf}: {e}")

    try:
        merger.write("madi_nekomath_combined.pdf")
        merger.close()
        logger.info("PDFs combinados con éxito en 'madi_nekomath_combined.pdf'")
    except Exception as e:
        logger.error(f"Error al escribir el PDF combinado: {e}")
else:
    logger.warning("No se crearon PDFs con éxito. No se puede combinar.")

logger.info("Proceso completado. Por favor, revisa los logs para más detalles.")

INFO:__main__:Contenido de la página: 

<!DOCTYPE html>


<html lang="es" >

  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="generator" content="Docutils 0.18.1: http://docutils.sourceforge.net/" />

    <title>Clases de complejidad y P vs. NP &#8212; MatemÃ¡ticas Discretas para Ciencia de Datos</title>
  
  
  
  <script data-cfasync="false">
    document.documentElement.dataset.mode = localStorage.getItem("mode") || "";
    document.documentElement.datas...
INFO:__main__:Enlace encontrado: #main-content
INFO:__main__:Enlace encontrado: ../intro.html
INFO:__main__:Enlace encontrado: ../intro.html
INFO:__main__:Enlace encontrado: ../P1/IntroCombinatoria.html
INFO:__main__:Enlace encontrado: ../P1/BuscarPatron.html
INFO:__main__:Enlace encontrado: ../P1/Casillas.html
INFO:__main__:Enlace encontrado: ../P1/DobleConteo.html
INFO:__main__:Enlace encontrado: ../P1/Induccion.html
INFO:__main__:Enlace encontrado: ..

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb1 in position 43516: invalid start byte

In [24]:
def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        logger.info(f"Contenido de la página: {response.text[:500]}...")
        soup = BeautifulSoup(response.text, 'html.parser')
        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            logger.info(f"Enlace encontrado: {href}")
            if href.endswith('.html') and not href.startswith(('#', 'http', 'https')):
                full_url = urllib.parse.urljoin(url, href)
                if full_url not in links:
                    links.append(full_url)
        logger.info(f"Total de enlaces relevantes encontrados: {len(links)}")
        return links
    except requests.exceptions.RequestException as e:
        logger.error(f"Error al obtener enlaces de la página principal: {e}")
        return []
    
def html_to_pdf(url, output_filename):
    try:
        # Paso 1: Descargar el contenido HTML
        response = requests.get(url)
        response.raise_for_status()
        html_content = response.text

        # Paso 2: Convertir HTML a LaTeX
        latex_filename = output_filename.replace('.pdf', '.tex')
        pandoc_html_to_latex = [
            'pandoc',
            '-f', 'html',
            '-t', 'latex',
            '--pdf-engine=xelatex',
            '--mathjax',
            '-V', 'geometry:margin=1in',
            '-V', 'mainfont:DejaVu Serif',
            '-V', 'monofont:DejaVu Sans Mono',
            '--standalone',
            '-o', latex_filename
        ]
        subprocess.run(pandoc_html_to_latex, input=html_content, text=True, check=True, capture_output=True)

        # Paso 3: Modificar el archivo LaTeX
        with open(latex_filename, 'r', encoding='utf-8') as file:
            latex_content = file.read()

        # Añadir paquetes y configuraciones adicionales
        latex_preamble = r"""
        \usepackage{fontspec}
        \usepackage{unicode-math}
        \setmainfont{DejaVu Serif}
        \setmonofont{DejaVu Sans Mono}
        \usepackage{amsmath}
        \usepackage{amssymb}
        \usepackage{graphicx}
        \usepackage{hyperref}
        \usepackage[spanish]{babel}
        """
        latex_content = latex_content.replace(r'\begin{document}', f'{latex_preamble}\n\\begin{{document}}')

        with open(latex_filename, 'w', encoding='utf-8') as file:
            file.write(latex_content)

        # Paso 4: Convertir LaTeX a PDF
        xelatex_command = ['xelatex', '-interaction=nonstopmode', latex_filename]
        for _ in range(2):  # Ejecutar dos veces para resolver referencias
            result = subprocess.run(xelatex_command, check=True, capture_output=True, text=True)
        
        logger.info(f"PDF creado con éxito: {output_filename}")

        # Limpiar archivos temporales
        for ext in ['.aux', '.log', '.out']:
            temp_file = output_filename.replace('.pdf', ext)
            if os.path.exists(temp_file):
                os.remove(temp_file)

        return True
    except subprocess.CalledProcessError as e:
        logger.error(f"Error en el proceso de conversión para {output_filename}: {e}")
        logger.error(f"Salida estándar: {e.stdout}")
        logger.error(f"Salida de error: {e.stderr}")
        return False
    except requests.exceptions.RequestException as e:
        logger.error(f"Error al descargar la página {url}: {e}")
        return False
    except Exception as e:
        logger.error(f"Error inesperado al crear PDF {output_filename}: {e}")
        return False

# Crear directorio para guardar los PDFs
if not os.path.exists("pdfs"):
    os.makedirs("pdfs")

# Obtener enlaces de la página principal
page_links = get_page_links(main_url)
logger.info(f"Enlaces obtenidos: {page_links}")

# Procesar cada página
successful_pdfs = []
for page_url in page_links:
    logger.info(f"Procesando página: {page_url}")
    
    pdf_filename = f"pdfs/{os.path.basename(page_url).replace('.html', '.pdf')}"
    if html_to_pdf(page_url, pdf_filename):
        successful_pdfs.append(pdf_filename)
    
    time.sleep(1)  # Esperar 1 segundo entre solicitudes

if successful_pdfs:
    merger = PdfMerger()
    for pdf in successful_pdfs:
        try:
            merger.append(pdf)
            logger.info(f"PDF agregado con éxito: {pdf}")
        except Exception as e:
            logger.error(f"Error al procesar {pdf}: {e}")

    try:
        merger.write("madi_nekomath_combined.pdf")
        merger.close()
        logger.info("PDFs combinados con éxito en 'madi_nekomath_combined.pdf'")
    except Exception as e:
        logger.error(f"Error al escribir el PDF combinado: {e}")
else:
    logger.warning("No se crearon PDFs con éxito. No se puede combinar.")

logger.info("Proceso completado. Por favor, revisa los logs para más detalles.")

INFO:__main__:Contenido de la página: 

<!DOCTYPE html>


<html lang="es" >

  <head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="generator" content="Docutils 0.18.1: http://docutils.sourceforge.net/" />

    <title>Clases de complejidad y P vs. NP &#8212; MatemÃ¡ticas Discretas para Ciencia de Datos</title>
  
  
  
  <script data-cfasync="false">
    document.documentElement.dataset.mode = localStorage.getItem("mode") || "";
    document.documentElement.datas...
INFO:__main__:Enlace encontrado: #main-content
INFO:__main__:Enlace encontrado: ../intro.html
INFO:__main__:Enlace encontrado: ../intro.html
INFO:__main__:Enlace encontrado: ../P1/IntroCombinatoria.html
INFO:__main__:Enlace encontrado: ../P1/BuscarPatron.html
INFO:__main__:Enlace encontrado: ../P1/Casillas.html
INFO:__main__:Enlace encontrado: ../P1/DobleConteo.html
INFO:__main__:Enlace encontrado: ../P1/Induccion.html
INFO:__main__:Enlace encontrado: ..

KeyboardInterrupt: 